In [33]:
# Script to determine the better performing ANN
# Check fitting

# fn to remove tf warnings when creating a keras model
def warn(*args, **kwargs):
    pass
    import warnings
    warnings.warn = warn

# import libraries
import numpy as np
import tensorflow as tf
import pandas as pd
# import pandas_profiling
# import seaborn as sns
# import matplotlib.pyplot as plt

from math import floor, ceil
# from pylab import rcParams
from tensorflow import keras

# eager execution enabled to execute operations immediately 
# without requiring a Session.run()
# tf.enable_eager_execution()

# %matplotlib inline

import codecs
import h5py

# importing py files
# from functions import sigmoid, sigmoid_backward, relu, relu_backward


# from __future__ import print_function

import numpy as np
# import pickle
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing import sequence
from keras.layers.noise import GaussianNoise
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution2D,MaxPooling2D
from keras.utils import np_utils

Using TensorFlow backend.


In [34]:
# set parameters

batch_size = 50
hidden_units = 10
nb_classes = 16
num_lstm_units = 350 #or 64?
num_epochs = 100

In [44]:
# prepare the data
mfcc_filename = "mfccs_dev_22050.csv"
dataset = pd.read_csv(mfcc_filename)
df = pd.DataFrame(dataset)
mfcc_data = df[["mfcc_mean1","mfcc_mean2","mfcc_mean3","mfcc_mean4","mfcc_mean5","mfcc_mean6","mfcc_mean7","mfcc_mean8","mfcc_mean9","mfcc_mean10","mfcc_mean11","mfcc_mean12","mfcc_mean13"]]
test_dim = mfcc_data.shape[0]

#set up targets (labels)
accent_data = pd.read_csv("cv-valid-dev-acc-mp3.csv")
targets_raw = np.array(accent_data['accent'])

label_encoder = LabelEncoder()
mfcc_targets = label_encoder.fit_transform(targets_raw)

X_train,X_test,y_train,y_test = train_test_split(
    mfcc_data,mfcc_targets,test_size=0.20,random_state=42
)

# X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
# X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1],1)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

1080 train sequences
270 test sequences
X_train shape: (1080, 13)
X_test shape: (270, 13)
y_train shape: (1080,)
y_test shape: (270,)


In [65]:
# feedforward neural network

# with softmax activation
ff_model = Sequential()

ff_model.add(Dense(4, kernel_regularizer=keras.regularizers.l2(0.001),
                       activation='softmax', input_dim=13)) 
ff_model.add(Dense(3, kernel_regularizer=keras.regularizers.l2(0.001),
                       activation='softmax'))
ff_model.add(Dense(nb_classes))

# define loss fn as the mean squared logarithmic error
# optimizaation fn is Adam
# The metrics are: mae->mean absolute error
#                  mse->mean squared error
#                  mape->mean absolute percentage error
#                  msle->mean squared logarithmic error
ff_model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy', 'mae', 'mse', 'mape'])

# epochs = 1000 gives the least mape in the range of 1 to 1000
# the greater the batch_size the greater the mape
# ff_model_history = ff_model.fit(X_train, Y_train,
#                                epochs=num_epochs,
#                                batch_size=batch_size,
# #                                validation_data=(X_test, Y_test),
#                                verbose=2)
ff_model.fit(X_train, Y_train,
             epochs=num_epochs, batch_size=batch_size,
             validation_data=(X_test, Y_test),
             verbose=2)

# predict the storage modulus using the test dataset
y_pred = ff_model.predict_classes(X_test, batch_size = batch_size)
print(classification_report(y_test, y_pred))

Train on 1080 samples, validate on 270 samples
Epoch 1/100
 - 0s - loss: 8.6878 - accuracy: 0.0037 - mae: 0.2219 - mse: 0.1012 - mape: 157875472.0000 - val_loss: 8.1378 - val_accuracy: 0.0074 - val_mae: 0.2260 - val_mse: 0.1033 - val_mape: 161286112.0000
Epoch 2/100
 - 0s - loss: 8.8460 - accuracy: 0.0037 - mae: 0.2228 - mse: 0.1017 - mape: 158516352.0000 - val_loss: 8.4370 - val_accuracy: 0.0074 - val_mae: 0.2260 - val_mse: 0.1035 - val_mape: 161175696.0000
Epoch 3/100
 - 0s - loss: 8.7813 - accuracy: 0.0037 - mae: 0.2230 - mse: 0.1019 - mape: 158585344.0000 - val_loss: 8.0751 - val_accuracy: 0.0074 - val_mae: 0.2265 - val_mse: 0.1038 - val_mape: 161557488.0000
Epoch 4/100
 - 0s - loss: 8.9253 - accuracy: 0.0037 - mae: 0.2231 - mse: 0.1019 - mape: 158641488.0000 - val_loss: 8.3675 - val_accuracy: 0.0074 - val_mae: 0.2260 - val_mse: 0.1038 - val_mape: 160935520.0000
Epoch 5/100
 - 0s - loss: 8.9960 - accuracy: 0.0046 - mae: 0.2229 - mse: 0.1019 - mape: 158381696.0000 - val_loss: 8.8005

KeyboardInterrupt: 

In [70]:
# generate the training and test samples 
from sklearn.model_selection import train_test_split

# feedforward neural network
# with ReLU activation
ff_model = keras.models.Sequential([
    keras.layers.Dense(4, kernel_regularizer=keras.regularizers.l2(0.001),
                       activation=tf.nn.softmax, input_dim=13),
    keras.layers.Dense(3, kernel_regularizer=keras.regularizers.l2(0.001),
                       activation=tf.nn.softmax),
    keras.layers.Dense(nb_classes)
])

# define loss fn as the mean squared logarithmic error
# optimizaation fn is Adam
# The metrics are: mae->mean absolute error
#                  mse->mean squared error
#                  mape->mean absolute percentage error
#                  msle->mean squared logarithmic error
ff_model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy', 'mae', 'mse', 'mape'])

# epochs = 1000 gives the least mape in the range of 1 to 1000
# the greater the batch_size the greater the mape
ff_model_history = ff_model.fit(X_train, Y_train,
                               epochs=num_epochs,
                               batch_size=batch_size,
                               validation_data=(X_test, Y_test),
                               verbose=2)

# predict the storage modulus using the test dataset
y_pred = ff_model.predict_classes(X_test, batch_size = batch_size)
print(classification_report(y_test, y_pred))

Train on 1080 samples, validate on 270 samples
Epoch 1/100
1080/1080 - 1s - loss: 10.6903 - accuracy: 0.0324 - mae: 0.2136 - mse: 0.0982 - mape: 152006832.0000 - val_loss: 9.3815 - val_accuracy: 0.0815 - val_mae: 0.2147 - val_mse: 0.0989 - val_mape: 152573584.0000
Epoch 2/100
1080/1080 - 0s - loss: 8.7258 - accuracy: 0.0602 - mae: 0.2161 - mse: 0.0998 - mape: 153467568.0000 - val_loss: 9.3212 - val_accuracy: 0.0889 - val_mae: 0.2158 - val_mse: 0.0998 - val_mape: 153134960.0000
Epoch 3/100
1080/1080 - 0s - loss: 8.7398 - accuracy: 0.0611 - mae: 0.2165 - mse: 0.1002 - mape: 153663024.0000 - val_loss: 9.5597 - val_accuracy: 0.0889 - val_mae: 0.2158 - val_mse: 0.0999 - val_mape: 153144416.0000
Epoch 4/100
1080/1080 - 0s - loss: 8.8438 - accuracy: 0.0620 - mae: 0.2165 - mse: 0.1002 - mape: 153643552.0000 - val_loss: 9.4998 - val_accuracy: 0.0889 - val_mae: 0.2158 - val_mse: 0.0998 - val_mape: 153115296.0000
Epoch 5/100
1080/1080 - 0s - loss: 9.3658 - accuracy: 0.0620 - mae: 0.2164 - mse: 0.

Epoch 39/100
1080/1080 - 0s - loss: 8.1682 - accuracy: 0.0713 - mae: 0.2167 - mse: 0.0995 - mape: 153012816.0000 - val_loss: 8.6010 - val_accuracy: 0.0889 - val_mae: 0.2163 - val_mse: 0.0994 - val_mape: 152820048.0000
Epoch 40/100
1080/1080 - 0s - loss: 8.0786 - accuracy: 0.0713 - mae: 0.2167 - mse: 0.0995 - mape: 152971504.0000 - val_loss: 8.6010 - val_accuracy: 0.0889 - val_mae: 0.2163 - val_mse: 0.0993 - val_mape: 152783072.0000
Epoch 41/100
1080/1080 - 0s - loss: 8.0189 - accuracy: 0.0713 - mae: 0.2166 - mse: 0.0995 - mape: 152907248.0000 - val_loss: 8.6009 - val_accuracy: 0.0889 - val_mae: 0.2163 - val_mse: 0.0993 - val_mape: 152741776.0000
Epoch 42/100
1080/1080 - 0s - loss: 8.0338 - accuracy: 0.0713 - mae: 0.2166 - mse: 0.0995 - mape: 152847376.0000 - val_loss: 8.5412 - val_accuracy: 0.0889 - val_mae: 0.2162 - val_mse: 0.0993 - val_mape: 152698432.0000
Epoch 43/100
1080/1080 - 0s - loss: 8.0188 - accuracy: 0.0713 - mae: 0.2165 - mse: 0.0995 - mape: 152786704.0000 - val_loss: 8.4

Epoch 77/100
1080/1080 - 0s - loss: 10.8835 - accuracy: 0.0796 - mae: 0.2158 - mse: 0.0987 - mape: 152343616.0000 - val_loss: 11.1671 - val_accuracy: 0.0889 - val_mae: 0.2157 - val_mse: 0.0986 - val_mape: 152417824.0000
Epoch 78/100
1080/1080 - 0s - loss: 10.8686 - accuracy: 0.0796 - mae: 0.2158 - mse: 0.0987 - mape: 152305904.0000 - val_loss: 11.1670 - val_accuracy: 0.0889 - val_mae: 0.2157 - val_mse: 0.0986 - val_mape: 152380208.0000
Epoch 79/100
1080/1080 - 0s - loss: 10.8835 - accuracy: 0.0806 - mae: 0.2158 - mse: 0.0986 - mape: 152269376.0000 - val_loss: 11.1670 - val_accuracy: 0.0889 - val_mae: 0.2156 - val_mse: 0.0986 - val_mape: 152342688.0000
Epoch 80/100
1080/1080 - 0s - loss: 10.8984 - accuracy: 0.0806 - mae: 0.2157 - mse: 0.0986 - mape: 152231856.0000 - val_loss: 11.1670 - val_accuracy: 0.0889 - val_mae: 0.2156 - val_mse: 0.0985 - val_mape: 152305472.0000
Epoch 81/100
1080/1080 - 0s - loss: 10.8984 - accuracy: 0.0806 - mae: 0.2157 - mse: 0.0986 - mape: 152195040.0000 - val_

/Users/yeojin_jung/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
